### Import Libraries:

In [1]:
# 1
import tensorflow as tf
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests

In [2]:
# 2
poetry_response = requests.get("https://raw.githubusercontent.com/Supriya-Suresh/Poetry-Generation-LSTM-Tensorflow/main/adele.txt")

In [3]:
# 3
print("Response Value: ", poetry_response.status_code, "\n")
poetry_response.text

Response Value:  200 



'Looking for some education\r\nMade my way into the night\r\nAll that bullshit conversation\r\nBaby, can\'t you read the signs? I won\'t bore you with the details, baby\r\nI don\'t even wanna waste your time\r\nLet\'s just say that maybe\r\nYou could help me ease my mind\r\nI ain\'t Mr. Right But if you\'re looking for fast love\r\nIf that\'s love in your eyes\r\nIt\'s more than enough\r\nHad some bad love\r\nSo fast love is all that I\'ve got on my mind Ooh, ooh\r\nOoh, ooh Looking for some affirmation\r\nMade my way into the sun\r\nMy friends got their ladies\r\nAnd they\'re all having babies\r\nI just wanna have some fun I won\'t bore you with the details, baby\r\nI don\'t even wanna waste your time\r\nLet\'s just say that maybe\r\nYou could help me ease my mind\r\nI ain\'t Mr. Right But if you\'re looking for fast love\r\nIf that\'s love in your eyes\r\nIt\'s more than enough\r\nI\'ve had some bad love\r\nSo fast love is all that I\'ve got on my mind Ooh, ooh\r\nBaby, baby\r\nOoh, 

### Split data based on new lines:

In [4]:
# 4
data = poetry_response.text.splitlines()
print("Length of Poetry list : ", len(data), "\n")
data[:11]

Length of Poetry list :  2400 



['Looking for some education',
 'Made my way into the night',
 'All that bullshit conversation',
 "Baby, can't you read the signs? I won't bore you with the details, baby",
 "I don't even wanna waste your time",
 "Let's just say that maybe",
 'You could help me ease my mind',
 "I ain't Mr. Right But if you're looking for fast love",
 "If that's love in your eyes",
 "It's more than enough",
 'Had some bad love']

### Word count:

In [5]:
# 5
len(" ".join(data))

91330

### Perform Tokenization:

In [6]:
# 6
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [7]:
# 7
tokenizer =  Tokenizer()
tokenizer.fit_on_texts(data)
print(type(tokenizer))
encoded_text = tokenizer.texts_to_sequences(data)
print(type(encoded_text))
print("Length of Encoded text data: ", len(encoded_text), "\n")
encoded_text[:11]

<class 'keras.preprocessing.text.Tokenizer'>
<class 'list'>
Length of Encoded text data:  2400 



[[254, 21, 219, 725],
 [117, 8, 80, 153, 3, 133],
 [14, 10, 726, 727],
 [41, 56, 2, 603, 3, 728, 1, 68, 517, 2, 40, 3, 518, 41],
 [1, 23, 107, 189, 300, 9, 57],
 [286, 35, 46, 10, 230],
 [2, 83, 134, 4, 519, 8, 120],
 [1, 37, 520, 102, 19, 27, 25, 254, 21, 328, 11],
 [27, 209, 11, 13, 9, 124],
 [42, 67, 210, 125],
 [49, 219, 446, 11]]

### Tokenizer details:

In [8]:
# 8
help(tokenizer)

Help on Tokenizer in module keras.preprocessing.text object:

class Tokenizer(builtins.object)
 |  Tokenizer(num_words=None, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ', char_level=False, oov_token=None, analyzer=None, **kwargs)
 |  
 |  Text tokenization utility class.
 |  
 |  Deprecated: `tf.keras.preprocessing.text.Tokenizer` does not operate on
 |  tensors and is not recommended for new code. Prefer
 |  `tf.keras.layers.TextVectorization` which provides equivalent functionality
 |  through a layer which accepts `tf.Tensor` input. See the
 |  [text loading tutorial](https://www.tensorflow.org/tutorials/load_data/text)
 |  for an overview of the layer and text handling in tensorflow.
 |  
 |  This class allows to vectorize a text corpus, by turning each
 |  text into either a sequence of integers (each integer being the index
 |  of a token in a dictionary) or into a vector where the coefficient
 |  for each token could be binary, based on word count, based

### Prepare Training Data:

In [9]:
# 9
# Adding 1 for adjustment in embedding:
vocab_size = len(tokenizer.word_counts) + 1
vocab_size

1396

In [10]:
# 10
data_list = []
for i in encoded_text:
  if len(i)>1:
    for j in range(2, len(i)):
      data_list.append(i[:j])

### Perform padding:

In [11]:
# 11
max_length = 20
sequences = pad_sequences(data_list, maxlen=max_length, padding="pre")
sequences.shape

(14231, 20)

In [12]:
# 12
X = sequences[:, :-1]
y = sequences[:, -1]
print(X.shape)
print(y.shape)

(14231, 19)
(14231,)


### One Hot Encoding:

In [13]:
# 13
y[:11]

array([ 21, 219,   8,  80, 153,   3,  10, 726,  56,   2, 603], dtype=int32)

In [14]:
# 14
y_cat = to_categorical(y, num_classes=vocab_size)
y_cat[:11]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [15]:
# 15
seq_length = X.shape[1]
seq_length

19

### LSTM Model creation:

In [16]:
# 16
model = Sequential()

model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [17]:
# 17
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 19, 50)            69800     
                                                                 
 lstm (LSTM)                 (None, 19, 100)           60400     
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 1396)              140996    
                                                                 
Total params: 361,696
Trainable params: 361,696
Non-trainable params: 0
_________________________________________________________________


In [21]:
# 18
model.fit(X, y_cat, batch_size=32, epochs=50)

Epoch 1/50
445/445 [==============================] - 19s 43ms/step - loss: 1.5616 - accuracy: 0.6116
Epoch 2/50
445/445 [==============================] - 19s 43ms/step - loss: 1.5254 - accuracy: 0.6205
Epoch 3/50
445/445 [==============================] - 19s 43ms/step - loss: 1.4946 - accuracy: 0.6286
Epoch 4/50
445/445 [==============================] - 19s 43ms/step - loss: 1.4724 - accuracy: 0.6325
Epoch 5/50
445/445 [==============================] - 19s 43ms/step - loss: 1.4404 - accuracy: 0.6383
Epoch 6/50
445/445 [==============================] - 19s 43ms/step - loss: 1.4143 - accuracy: 0.6484
Epoch 7/50
445/445 [==============================] - 19s 42ms/step - loss: 1.3914 - accuracy: 0.6456
Epoch 8/50
445/445 [==============================] - 19s 42ms/step - loss: 1.3541 - accuracy: 0.6600
Epoch 9/50
445/445 [==============================] - 19s 42ms/step - loss: 1.3361 - accuracy: 0.6595
Epoch 10/50
445/445 [==============================] - 19s 43ms/step - loss: 1.312

### Poetry Generation:

In [22]:
# 19
poetry_length = 10

def poetry_generator(seed_text, n_lines):

  for i in range(n_lines):
    text = []
    for _ in range(poetry_length):
      encoded = tokenizer.texts_to_sequences([seed_text])
      encoded_data = pad_sequences(encoded, maxlen=seq_length, padding="pre")

      y_pred = np.argmax(model.predict(encoded_data), axis=-1)

      predicted_word = ""

      for word, index in tokenizer.word_index.items():
        if index == y_pred:
          predicted_word = word
          break

      seed_text = seed_text + " " + predicted_word
      text.append(predicted_word)

    seed_text = text[-1]
    text = " ".join(text)
    print(text)

In [23]:
# 20
seed_text = "love me like you do"
poetry_generator(seed_text, 5)

1/1 [==============================] - 0s 20ms/step
is on your name when was a million years ago
1/1 [==============================] - 0s 20ms/step
you swear you you should i give you everything that
1/1 [==============================] - 0s 21ms/step
i never knew never love you leave my heart around
1/1 [==============================] - 0s 23ms/step
had it in hold me like i'm more than just
1/1 [==============================] - 0s 21ms/step
'cause you heard it rumor has it down you a
